In [1]:
from collections import namedtuple
from itertools import chain, groupby
import csv
import numpy

In [2]:
Match = namedtuple("Match", ['gameId', 'date', 'time', 'team1', 'team2', 'group'])
Prediction = namedtuple("Prediction", ['team1', 'team2', 'outcome', 'sd'])
Team = namedtuple("Team", ['name', 'draw'])
Result = namedtuple("Result", ['match', 'team1Score', 'team2Score'])
TeamResult = namedtuple("TeamResult", ['name', 'opponent', 'group', 'gs', 'ga'])
TeamRecord = namedtuple("TeamRecord", ['name', 'group', 'p', 'gs', 'ga', 'defeated'])

In [3]:
def loadCsv(filepath, itemtype):
    with open(filepath, 'r') as csvfile:
        reader = csv.DictReader(csvfile, delimiter=',')
        return list(map(lambda row: itemtype(**row), reader))

In [4]:
def getActualGroupMatch(match, teams):
    return Match(gameId = match.gameId, date = match.date, time = match.time, group = match.group,
        team1 = next(team.name for team in teams if team.draw == match.team1),
        team2 = next(team.name for team in teams if team.draw == match.team2),       
        )

In [5]:
def getActualEliminationMatch(match, qualified):
    return Match(gameId = match.gameId, date = match.date, time = match.time, group = match.group,
        team1 = qualified[match.team1],
        team2 = qualified[match.team2],
        )

In [6]:
def getResult(match, predictions):
    allowtie = match.group != None 
    prediction = next(pred for pred in predictions \
        if (pred.team1 == match.team1 and pred.team2 == match.team2) or \
            (pred.team1 == match.team2 and pred.team2 == match.team1)) 
    reverse = prediction.team1 != match.team1
    goaldiff = numpy.random.normal(float(prediction.outcome), float(prediction.sd))
    reverse = reverse != (goaldiff < 0) 
    normgoaldiff = 0 if abs(goaldiff) <= 0.4475 \
        else 1 if abs(goaldiff) < 1 else round(abs(goaldiff))
    if not(allowtie) and normgoaldiff == 0:
        normgoaldiff = 1
    losergoals = round(abs(numpy.random.normal(0, 1)))
    return Result(match, losergoals, losergoals + normgoaldiff) if reverse \
        else Result(match, losergoals + normgoaldiff, losergoals)

In [7]:
def getTeamPerfFromResult(result):
    return [TeamResult(name=result.match.team1, opponent=result.match.team2, group=result.match.group, gs=result.team1Score, ga=result.team2Score),
    TeamResult(name=result.match.team2, opponent=result.match.team1, group=result.match.group, gs=result.team2Score, ga=result.team1Score)]

In [8]:
def getTeamRecord(name, teamResults):
    points = w = d = gs = ga = 0
    group = ''
    defeated = []
    for result in teamResults:
        group = result.group
        gs += result.gs
        ga += result.ga
        if result.gs > result.ga:
            w += 1
            defeated.append(result.opponent)
        elif result.gs == result.ga:
            d += 1
    return TeamRecord(name=name, group=group, p=3*w+d, gs=gs, ga=ga, defeated=defeated)

In [9]:
def runGroupStage(schedule, predictions, teams):
    groupschedule = list(map(lambda x: getActualGroupMatch(x, teams), filter(lambda x: x.group != None, schedule)))
    groupresults = list(map(lambda x: getResult(x, predictions), groupschedule))
    groupperfs = list(chain.from_iterable(map(getTeamPerfFromResult, groupresults)))
    records = []
    for k, g in groupby(sorted(groupperfs, key=lambda x: x.name), key=lambda x: x.name):
        records.append(getTeamRecord(k, g))
    qualified = {}
    group_standings = {}
    group_table_total = {}
    for k, g in groupby(sorted(records, key=lambda x: x.group), key=lambda x: x.group):
        grouptable = sorted(g, key=lambda x: 1000000*x.p + 10000*(x.gs-x.ga) + 100*(x.gs), reverse=True)
        qualified['1' + k] = grouptable[0].name
        qualified['2' + k] = grouptable[1].name
        
        group_standings['1' + k] = grouptable[0].name
        group_standings['2' + k] = grouptable[1].name
        group_standings['3' + k] = grouptable[2].name
        group_standings['4' + k] = grouptable[3].name
        
        
        
        group_table_standings = []
        
        group_table_standings.append([grouptable[0].name, grouptable[0].gs, grouptable[0].ga, grouptable[0].p])
        group_table_standings.append([grouptable[1].name, grouptable[1].gs, grouptable[1].ga, grouptable[1].p])
        group_table_standings.append([grouptable[2].name, grouptable[2].gs, grouptable[2].ga, grouptable[2].p])
        group_table_standings.append([grouptable[3].name, grouptable[3].gs, grouptable[3].ga, grouptable[3].p])
        
        group_table_total[grouptable[0].group] = group_table_standings
    return qualified, group_standings, groupresults, group_table_total

In [10]:
def runEliminationRound(round, prevround, schedule, predictions):
    roundschedule = list(map(lambda x: getActualEliminationMatch(x, prevround), filter(lambda x: x.gameId.startswith('G'+round), schedule)))
    roundresults = list(map(lambda x: getResult(x, predictions), roundschedule))
    results = {}
    for result in roundresults:
        results['W' + result.match.gameId] = result.match.team1 if result.team1Score > result.team2Score \
            else result.match.team2
        results['L' + result.match.gameId] = result.match.team1 if result.team1Score < result.team2Score \
            else result.match.team2
    return results, roundresults

In [11]:
schedule = loadCsv('fifawm/tournament/data/schedule_fifa_fs.csv', Match)
predictions = loadCsv('data/wc2018staticPredictions.csv', Prediction)
teams = loadCsv('data/wc2018qualified.csv', Team)

In [12]:
with open("simresultsasd.csv", "w+") as f:
#     f.write('iteration,winner\n')
    for i in range(1):
#     for i in range(10000):
        results, group_standings, groupresults, group_table_total = runGroupStage(schedule, predictions, teams)
        asd = results
        all_rounds = []
        for rnd in ['AF','QF','SF','FN']:
            results, roundresults = runEliminationRound(rnd, results, schedule, predictions)
            all_rounds.append(roundresults)

#         f.write(str(i+1) + ',' + results['WGFN2'] + '\n')

In [13]:
group_table_total

{'A': [['URU', 6, 3, 6], ['RUS', 4, 4, 6], ['KSA', 6, 7, 4], ['EGY', 2, 4, 1]],
 'B': [['POR', 8, 2, 7],
  ['IRN', 9, 6, 6],
  ['ESP', 5, 6, 4],
  ['MAR', 4, 12, 0]],
 'C': [['FRA', 9, 5, 6],
  ['AUS', 11, 8, 6],
  ['PER', 3, 6, 3],
  ['DEN', 5, 9, 3]],
 'D': [['ARG', 11, 2, 9],
  ['CRO', 2, 2, 4],
  ['NGA', 1, 6, 2],
  ['ISL', 2, 6, 1]],
 'E': [['BRA', 3, 1, 7], ['SRB', 4, 2, 6], ['SUI', 4, 3, 3], ['CRC', 3, 8, 1]],
 'F': [['GER', 8, 3, 7], ['SWE', 4, 3, 5], ['KOR', 2, 5, 2], ['MEX', 4, 7, 1]],
 'G': [['ENG', 7, 3, 6], ['BEL', 5, 4, 6], ['TUN', 4, 6, 3], ['PAN', 3, 6, 3]],
 'H': [['SEN', 7, 3, 7], ['POL', 4, 3, 4], ['JPN', 3, 5, 2], ['COL', 2, 5, 2]]}

In [14]:
import pandas as pd

In [15]:
columns = ['date', 'team1', 'team2', 'group', 'result']
new_df = pd.DataFrame(columns=columns)

In [16]:
columns = ['gameID', 'date', 'team1', 'team2', 'group', 'result']
new_df = pd.DataFrame(columns=columns)
for match in groupresults:
    new_df = new_df.append(pd.DataFrame([[match.match.gameId,
                                          match.match.date, 
                                          match.match.team1, 
                                          match.match.team2, 
                                          match.match.group, 
                                          '{}-{}'.format(match.team1Score, 
                                                         match.team2Score)]], 
                                         columns=columns))
    

for round in all_rounds:
    for match in round:
        new_df = new_df.append(pd.DataFrame([[match.match.gameId,
                                              match.match.date, 
                                              match.match.team1, 
                                              match.match.team2, 
                                              match.match.group, 
                                              '{}-{}'.format(match.team1Score, 
                                                             match.team2Score)]], 
                                             columns=columns))

In [17]:
data = open('simresults.csv').readlines()

In [18]:
winners = [line.strip().split(',')[1] for line in data[1:]]

In [19]:
from collections import Counter

In [20]:
Counter(winners).most_common()

[('BRA', 1248),
 ('ARG', 1101),
 ('GER', 952),
 ('RUS', 887),
 ('ENG', 709),
 ('POR', 687),
 ('FRA', 614),
 ('ESP', 613),
 ('SUI', 536),
 ('CRO', 405),
 ('BEL', 381),
 ('DEN', 223),
 ('COL', 193),
 ('SRB', 175),
 ('URU', 168),
 ('MEX', 156),
 ('POL', 105),
 ('SWE', 104),
 ('KOR', 99),
 ('AUS', 91),
 ('IRN', 87),
 ('JPN', 86),
 ('KSA', 81),
 ('CRC', 64),
 ('NGA', 57),
 ('SEN', 36),
 ('TUN', 32),
 ('PAN', 31),
 ('EGY', 26),
 ('MAR', 22),
 ('ISL', 16),
 ('PER', 15)]

In [21]:
schedule_fifa_sm = open('worldcup/static/data/schedule_fifa.csv').readlines()

In [22]:
schedule_fifa_sm = [line.strip() for line in schedule_fifa_sm[1:-16]]

In [23]:
teams = open('data/teams.csv').readlines()

In [24]:
teams = {team.split(',')[1]:team.split(',')[2] for team in teams[1:]}

In [25]:
groupss = open('data/wc2018qualified.csv').readlines()

In [26]:
groupss = [line.strip() for line in groupss[1:]]

In [27]:
groupss = {elem.split(',')[0]:elem.split(',')[1] for elem in groupss}

In [28]:
#gameId,date,time,team1,team2,group

In [29]:
gid_counter = {'A': 0, 'B' : 0, 'C' : 0, 'D' : 0, 'E' : 0, 'F' : 0, 'G' : 0, 'H' : 0}
with open('schedule_fifa_fs.csv', 'w') as out_file:
    for match in schedule_fifa_sm:
        elems = match.split(',')
#         if elems[0] is in ['123']:
        group = elems[-2].split(' ')[-1]
        gid_counter[group] = gid_counter[group] + 1
        
        gameID = 'G{}{}'.format(group, gid_counter[group])
        date = elems[1].split(' ')[0]
        time = elems[1].split(' ')[1]
        team1 = groupss[teams[elems[-3]]]
        team2 = groupss[teams[elems[-4]]]
        
        out_file.write('{},{},{},{},{},{}\n'.format(gameID, date, time, team1, team2, group))
    
    

In [30]:
groupss[teams[elems[-3]]]

'G1'